In [314]:
import pandas as pd
import numpy as np

In [315]:
df=pd.read_csv('./개인타자.csv',index_col=0)

In [316]:
df.columns

Index(['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'P_ID',
       'START_CK', 'BAT_ORDER_NO', 'PA', 'AB', 'RBI', 'RUN', 'HIT', 'H2', 'H3',
       'HR', 'SB', 'CS', 'SH', 'SF', 'BB', 'IB', 'HP', 'KK', 'GD', 'ERR',
       'LOB', 'P_HRA_RT', 'P_AB_CN', 'P_HIT_CN', 'GYEAR', 'NAME', 'POSITION',
       'AGE_VA', 'MONEY', 'GWEEK', 'STADIUM', 'HOME'],
      dtype='object')

In [317]:
df

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,P_ID,START_CK,BAT_ORDER_NO,PA,...,P_AB_CN,P_HIT_CN,GYEAR,NAME,POSITION,AGE_VA,MONEY,GWEEK,STADIUM,HOME
0,20160401HHLG0,20160401,HH,LG,0,T,60404,0,3,1,...,0,0,2016,송주호,외,28.0,4000만원,금,잠실,0
1,20160401HHLG0,20160401,LG,HH,0,B,61102,1,8,3,...,0,0,2016,유강남,포,24.0,8100만원,금,잠실,1
2,20160401HHLG0,20160401,LG,HH,0,B,61186,1,7,4,...,1,1,2016,이천웅,외,28.0,2800만원,금,잠실,1
3,20160401HHLG0,20160401,LG,HH,0,B,62164,0,9,1,...,0,0,2016,서상우,내,27.0,6000만원,금,잠실,1
4,20160401HHLG0,20160401,HH,LG,0,T,62700,1,9,2,...,0,0,2016,하주석,내,22.0,3200만원,금,잠실,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81097,20200719WOSK0,20200719,SK,WO,0,B,75847,1,3,4,...,1,0,2020,최정,내,33.0,120000만원,일,문학,1
81098,20200719WOSK0,20200719,SK,WO,0,B,76802,1,9,4,...,1,1,2020,김성현,내,33.0,21000만원,일,문학,1
81099,20200719WOSK0,20200719,SK,WO,0,B,77463,0,6,1,...,0,0,2020,채태인,내,38.0,10000만원,일,문학,1
81100,20200719WOSK0,20200719,WO,SK,0,T,78168,1,1,5,...,1,0,2020,서건창,내,31.0,35000만원,일,문학,0


In [318]:
df[['PA', 'AB', 'RBI', 'RUN', 'HIT', 'H2', 'H3',
       'HR', 'SB', 'CS', 'SH', 'SF', 'BB', 'IB', 'HP', 'KK', 'GD', 'ERR',
       'LOB', 'P_HRA_RT', 'P_AB_CN', 'P_HIT_CN']]=df.groupby(['P_ID','GYEAR'],as_index=False)[['PA', 'AB', 'RBI', 'RUN', 'HIT', 'H2', 'H3',
       'HR', 'SB', 'CS', 'SH', 'SF', 'BB', 'IB', 'HP', 'KK', 'GD', 'ERR',
       'LOB', 'P_HRA_RT', 'P_AB_CN', 'P_HIT_CN']].cumsum()

In [319]:
df1=df

## OBP

In [320]:
df1['H1']=df1['HIT']-(df1['H2']+df1['H3']+df1['HR'])

In [321]:
df1['OBP']=(df1['HIT']+df1['BB']+df1['HP'])/(df1['AB']+df1['BB']+df1['HP']+df1['SF'])

In [322]:
df1['OBP']

0        1.000000
1        0.000000
2        0.750000
3        1.000000
4        0.500000
           ...   
81097    0.413655
81098    0.314685
81099    0.360656
81100    0.395683
81101    0.380645
Name: OBP, Length: 81102, dtype: float64

## SLG

(단타 수 + 2루타 수*2 + 3루타 수*3 + 홈런 수*4)/타수 = 루타수 / 타수

In [360]:
df1['SLG']=(df1['HIT']+2*df1['H2']+3*df1['H3']+4*df1['HR'])/df1['AB']

In [361]:
df1['SLG']

0        1.000000
1        0.000000
2        1.750000
3             NaN
4        0.500000
           ...   
81097    0.666667
81098    0.251969
81099    0.446429
81100    0.521368
81101    0.428571
Name: SLG, Length: 81102, dtype: float64

## OPS

출루율 + 장타율

In [379]:
df1['OPS']=df1['OBP']+df1['SLG']

## wOBA

<img src="https://t1.daumcdn.net/cfile/tistory/2475254754FB25A635">

분모를 PA-IBB로 계산했을 때 다른 사이트에서 계산한 값이랑 비슷해지는 것 같기도..

In [380]:
import requests
from bs4 import BeautifulSoup

req = requests.get('http://www.statiz.co.kr/constant.php')
html = req.text
soup = BeautifulSoup(html, 'html.parser')
tables = soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div.col-md-12.col-xs-12.col-sm-12.col-lg-9 > div > div > div > div.box-body.no-padding.table-responsive > table')
table = tables[0]


In [381]:
table_html=str(table)
table_df_list=pd.read_html(table_html)
table_df = table_df_list[0]

In [385]:
idx=table_df[0:5]

In [386]:
idx.rename(columns = {'연도':'GYEAR'}, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [387]:
idx=idx[['GYEAR','wBB','wHBP','w1B','w2B','w3B','wHR']]
idx

,GYEAR,wBB,wHBP,w1B,w2B,w3B,wHR
0,2020,0.729,0.758,0.936,1.280,1.590,1.951
1,2019,0.721,0.752,0.945,1.317,1.653,2.074
2,2018,0.726,0.753,0.920,1.242,1.533,1.857
3,2017,0.728,0.756,0.927,1.259,1.558,1.901
4,2016,0.747,0.774,0.944,1.273,1.569,1.898


In [388]:
df1

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,P_ID,START_CK,BAT_ORDER_NO,PA,...,POSITION,AGE_VA,MONEY,GWEEK,STADIUM,HOME,H1,OBP,SLG,OPS
0,20160401HHLG0,20160401,HH,LG,0,T,60404,0,3,1,...,외,28.0,4000만원,금,잠실,0,1,1.000000,1.000000,2.000000
1,20160401HHLG0,20160401,LG,HH,0,B,61102,1,8,3,...,포,24.0,8100만원,금,잠실,1,0,0.000000,0.000000,0.000000
2,20160401HHLG0,20160401,LG,HH,0,B,61186,1,7,4,...,외,28.0,2800만원,금,잠실,1,2,0.750000,1.750000,2.500000
3,20160401HHLG0,20160401,LG,HH,0,B,62164,0,9,1,...,내,27.0,6000만원,금,잠실,1,0,1.000000,NaN,NaN
4,20160401HHLG0,20160401,HH,LG,0,T,62700,1,9,2,...,내,22.0,3200만원,금,잠실,0,1,0.500000,0.500000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81097,20200719WOSK0,20200719,SK,WO,0,B,75847,1,3,249,...,내,33.0,120000만원,일,문학,1,32,0.413655,0.666667,1.080321
81098,20200719WOSK0,20200719,SK,WO,0,B,76802,1,9,149,...,내,33.0,21000만원,일,문학,1,29,0.314685,0.251969,0.566654
81099,20200719WOSK0,20200719,SK,WO,0,B,77463,0,6,61,...,내,38.0,10000만원,일,문학,1,15,0.360656,0.446429,0.807084
81100,20200719WOSK0,20200719,WO,SK,0,T,78168,1,1,278,...,내,31.0,35000만원,일,문학,0,50,0.395683,0.521368,0.917051


In [389]:
data=pd.merge(df1,idx,on='GYEAR')
data

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,P_ID,START_CK,BAT_ORDER_NO,PA,...,H1,OBP,SLG,OPS,wBB,wHBP,w1B,w2B,w3B,wHR
0,20160401HHLG0,20160401,HH,LG,0,T,60404,0,3,1,...,1,1.000000,1.000000,2.000000,0.747,0.774,0.944,1.273,1.569,1.898
1,20160401HHLG0,20160401,LG,HH,0,B,61102,1,8,3,...,0,0.000000,0.000000,0.000000,0.747,0.774,0.944,1.273,1.569,1.898
2,20160401HHLG0,20160401,LG,HH,0,B,61186,1,7,4,...,2,0.750000,1.750000,2.500000,0.747,0.774,0.944,1.273,1.569,1.898
3,20160401HHLG0,20160401,LG,HH,0,B,62164,0,9,1,...,0,1.000000,NaN,NaN,0.747,0.774,0.944,1.273,1.569,1.898
4,20160401HHLG0,20160401,HH,LG,0,T,62700,1,9,2,...,1,0.500000,0.500000,1.000000,0.747,0.774,0.944,1.273,1.569,1.898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81097,20200719WOSK0,20200719,SK,WO,0,B,75847,1,3,249,...,32,0.413655,0.666667,1.080321,0.729,0.758,0.936,1.280,1.590,1.951
81098,20200719WOSK0,20200719,SK,WO,0,B,76802,1,9,149,...,29,0.314685,0.251969,0.566654,0.729,0.758,0.936,1.280,1.590,1.951
81099,20200719WOSK0,20200719,SK,WO,0,B,77463,0,6,61,...,15,0.360656,0.446429,0.807084,0.729,0.758,0.936,1.280,1.590,1.951
81100,20200719WOSK0,20200719,WO,SK,0,T,78168,1,1,278,...,50,0.395683,0.521368,0.917051,0.729,0.758,0.936,1.280,1.590,1.951


In [390]:
data['wOBA']=(data['wBB']*(data['BB']-data['IB'])+data['wHBP']*data['HP']+data['w1B']*data['H1']+data['w2B']*data['H2']+data['w3B']*data['H3']+data['wHR']*data['HR'])/(data['AB']+data['BB']-data['IB']+data['SF']+data['HP'])

In [391]:
data['wOBA2']=(data['wBB']*(data['BB']-data['IB'])+data['wHBP']*data['HP']+data['w1B']*data['H1']+data['w2B']*data['H2']+data['w3B']*data['H3']+data['wHR']*data['HR'])/(data['PA']-data['IB'])

In [392]:
data

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,P_ID,START_CK,BAT_ORDER_NO,PA,...,SLG,OPS,wBB,wHBP,w1B,w2B,w3B,wHR,wOBA,wOBA2
0,20160401HHLG0,20160401,HH,LG,0,T,60404,0,3,1,...,1.000000,2.000000,0.747,0.774,0.944,1.273,1.569,1.898,0.944000,0.944000
1,20160401HHLG0,20160401,LG,HH,0,B,61102,1,8,3,...,0.000000,0.000000,0.747,0.774,0.944,1.273,1.569,1.898,0.000000,0.000000
2,20160401HHLG0,20160401,LG,HH,0,B,61186,1,7,4,...,1.750000,2.500000,0.747,0.774,0.944,1.273,1.569,1.898,0.946500,0.946500
3,20160401HHLG0,20160401,LG,HH,0,B,62164,0,9,1,...,NaN,NaN,0.747,0.774,0.944,1.273,1.569,1.898,0.747000,0.747000
4,20160401HHLG0,20160401,HH,LG,0,T,62700,1,9,2,...,0.500000,1.000000,0.747,0.774,0.944,1.273,1.569,1.898,0.472000,0.472000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81097,20200719WOSK0,20200719,SK,WO,0,B,75847,1,3,249,...,0.666667,1.080321,0.729,0.758,0.936,1.280,1.590,1.951,0.420532,0.420532
81098,20200719WOSK0,20200719,SK,WO,0,B,76802,1,9,149,...,0.251969,0.566654,0.729,0.758,0.936,1.280,1.590,1.951,0.275643,0.264544
81099,20200719WOSK0,20200719,SK,WO,0,B,77463,0,6,61,...,0.446429,0.807084,0.729,0.758,0.936,1.280,1.590,1.951,0.347633,0.347633
81100,20200719WOSK0,20200719,WO,SK,0,T,78168,1,1,278,...,0.521368,0.917051,0.729,0.758,0.936,1.280,1.590,1.951,0.381299,0.381299


In [395]:
data[data['P_ID']==60404][['P_ID','GYEAR','OBP','SLG','OPS','wOBA','wOBA2']]

,P_ID,GYEAR,OBP,SLG,OPS,wOBA,wOBA2
0,60404,2016,1.000000,1.000000,2.000000,0.944000,0.944000
123,60404,2016,0.333333,0.333333,0.666667,0.314667,0.314667
402,60404,2016,0.200000,0.200000,0.400000,0.188800,0.188800
621,60404,2016,0.222222,0.222222,0.444444,0.209778,0.209778
643,60404,2016,0.181818,0.181818,0.363636,0.171636,0.171636
887,60404,2016,0.181818,0.181818,0.363636,0.171636,0.171636
1110,60404,2016,0.166667,0.166667,0.333333,0.157333,0.157333
3083,60404,2016,0.200000,0.142857,0.342857,0.175667,0.164688
3157,60404,2016,0.176471,0.125000,0.301471,0.155000,0.146389
3286,60404,2016,0.166667,0.117647,0.284314,0.146389,0.138684


## WPA

## BABIP

 (총 안타수-홈런)/(타수-삼진-홈런+희생플라이)

In [377]:
data['BABIP']=(data['HIT']-data['HR'])/(data['AB']-data['KK']-data['HR']+data['SF'])

In [378]:
data

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,P_ID,START_CK,BAT_ORDER_NO,PA,...,SLG,wBB,wHBP,w1B,w2B,w3B,wHR,wOBA,wOBA2,BABIP
0,20160401HHLG0,20160401,HH,LG,0,T,60404,0,3,1,...,1.000000,0.747,0.774,0.944,1.273,1.569,1.898,0.944000,0.944000,1.000000
1,20160401HHLG0,20160401,LG,HH,0,B,61102,1,8,3,...,0.000000,0.747,0.774,0.944,1.273,1.569,1.898,0.000000,0.000000,0.000000
2,20160401HHLG0,20160401,LG,HH,0,B,61186,1,7,4,...,1.750000,0.747,0.774,0.944,1.273,1.569,1.898,0.946500,0.946500,1.000000
3,20160401HHLG0,20160401,LG,HH,0,B,62164,0,9,1,...,NaN,0.747,0.774,0.944,1.273,1.569,1.898,0.747000,0.747000,NaN
4,20160401HHLG0,20160401,HH,LG,0,T,62700,1,9,2,...,0.500000,0.747,0.774,0.944,1.273,1.569,1.898,0.472000,0.472000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81097,20200719WOSK0,20200719,SK,WO,0,B,75847,1,3,249,...,0.666667,0.729,0.758,0.936,1.280,1.590,1.951,0.420532,0.420532,0.294118
81098,20200719WOSK0,20200719,SK,WO,0,B,76802,1,9,149,...,0.251969,0.729,0.758,0.936,1.280,1.590,1.951,0.275643,0.264544,0.256410
81099,20200719WOSK0,20200719,SK,WO,0,B,77463,0,6,61,...,0.446429,0.729,0.758,0.936,1.280,1.590,1.951,0.347633,0.347633,0.340909
81100,20200719WOSK0,20200719,WO,SK,0,T,78168,1,1,278,...,0.521368,0.729,0.758,0.936,1.280,1.590,1.951,0.381299,0.381299,0.313725
